# Calibrate the two cameras

In [18]:
# from https://github.com/niconielsen32/ComputerVision/blob/master/cameraCalibration.py
# modified
import numpy as np
import cv2 as cv
import glob

img_path = 'assets/calibration/'

################ FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS #############################

chessboardSize = (7,7)
frameSize = (1600,1200)

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

def calibrate(file_selector, index, out_path):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
    objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

    size_of_chessboard_squares_mm = 20
    objp = objp * size_of_chessboard_squares_mm


    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.


    images = glob.glob(img_path + file_selector + "-*.jpg")

    for image in images:

        img = cv.imread(image)
        img = cv.resize(img, frameSize)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        # Find the chess board corners
        ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)

        # If found, add object points, image points (after refining them)
        print(ret)
        if ret == True:

            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners)


    ############## CALIBRATION #######################################################

    ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

    #print("ImgPoints: ", imgpoints)

    print('')

    print("Camera calibrated: ", ret)
    print("Camera Matrix: ", cameraMatrix)
    print("Distortion Parameters: ", dist)
    print("Rotation Vectors: ", rvecs)
    print("Translation Vectors: ", tvecs)
    
    # ===========================  Calculate P  ==============================
    R = cv.Rodrigues(rvecs[index])[0]
    t = tvecs[index]
    W = np.concatenate([R,t], axis=-1) # [R|t]
    K = cameraMatrix
    P = K.dot(W)
    print('P for '+ file_selector + str(index))
    print(P)


    ############## UNDISTORTION #####################################################

    #img = cv.imread(img_path + file_selector + "-0" + str(index+1) + ".jpg")
    #h,  w = img.shape[:2]
    #newCameraMatrix, roi = cv.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))


    # Undistort
    #dst = cv.undistort(img, cameraMatrix, dist, None, newCameraMatrix)

    # crop the image
    #x, y, w, h = roi
    #dst = dst[y:y+h, x:x+w]
    #cv.imwrite(out_path, dst)



# Undistort with Remapping
#mapx, mapy = cv.initUndistortRectifyMap(cameraMatrix, dist, None, newCameraMatrix, (w,h), 5)
#dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# crop the image
#x, y, w, h = roi
#dst = dst[y:y+h, x:x+w]
#cv.imwrite('caliResult2.png', dst)




# Reprojection Error
#mean_error = 0

#for i in range(len(objpoints)):
#    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
#    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
#    mean_error += error

#print( "total error: {}".format(mean_error/len(objpoints)) )


In [19]:
calibrate("left", 3, "assets/left.jpg")
calibrate("right", 3, "assets/right.jpg")

True
True
True
True
True
True
True
True
True
True

Camera calibrated:  1.164967173339667
Camera Matrix:  [[977.49403966   0.         921.89633554]
 [  0.         976.28757876 601.63782338]
 [  0.           0.           1.        ]]
Distortion Parameters:  [[-0.31624916  0.53465289  0.03410252 -0.01014613 -0.43775636]]
Rotation Vectors:  (array([[0.0467419 ],
       [0.1320791 ],
       [0.84696031]]), array([[-0.04156076],
       [ 0.04312627],
       [ 0.32179524]]), array([[0.06146839],
       [1.02553077],
       [1.11293121]]), array([[-0.28076198],
       [ 0.34400388],
       [ 1.37047687]]), array([[ 0.05405482],
       [-0.00875938],
       [ 0.09993834]]), array([[ 0.02668436],
       [-0.09062509],
       [ 0.27041739]]), array([[-0.1192017 ],
       [-0.32119431],
       [ 0.05870834]]), array([[0.0104439 ],
       [0.03942026],
       [3.12825668]]), array([[-0.03152377],
       [-0.00508913],
       [ 0.08202337]]), array([[-0.02726709],
       [ 0.04254648],
       [ 1.08

In [50]:
Pp = np.matrix([[ 6.12430486e+02, -9.29557380e+02,  8.04197963e+02,
         9.98847541e+05],
       [ 8.35836177e+02,  5.46965600e+02,  6.29515656e+02,
         4.76432546e+05],
       [ 3.87999651e-04, -1.16635373e-01,  9.93174727e-01,
         7.88766677e+02]])
P = np.matrix([[-2.07810101e+02, -9.57532647e+02,  9.19420275e+02,
         4.99058538e+05],
       [ 6.41846643e+02,  1.77844561e+02,  9.33546842e+02,
         6.25080797e+05],
       [-3.98103114e-01,  4.43355772e-03,  9.17329959e-01,
         7.76363781e+02]])



print("P\n",P)
print("Pp\n",Pp)

P
 [[-2.07810101e+02 -9.57532647e+02  9.19420275e+02  4.99058538e+05]
 [ 6.41846643e+02  1.77844561e+02  9.33546842e+02  6.25080797e+05]
 [-3.98103114e-01  4.43355772e-03  9.17329959e-01  7.76363781e+02]]
Pp
 [[ 6.12430486e+02 -9.29557380e+02  8.04197963e+02  9.98847541e+05]
 [ 8.35836177e+02  5.46965600e+02  6.29515656e+02  4.76432546e+05]
 [ 3.87999651e-04 -1.16635373e-01  9.93174727e-01  7.88766677e+02]]
